<a href="https://colab.research.google.com/github/Rizukaf-id/DRPM-machine-learning/blob/main/DRPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re

## Load Data

In [3]:
file = "/content/drive/MyDrive/DRPM 2024/code/data/after_preprocessed_data_with_emoji.csv"
df = pd.read_csv(file)
df

,text,anotasi_gpt,emoji_berulang,new text,cleaning,case_folding,stopword_removal,stemming
0,",Sama bunda juga sedih ,mereka tulang punggung...",Non-Sarcasm,False,",Sama bunda juga sedih ,mereka tulang punggung...",Sama bunda juga sedih mereka tulang punggung ...,sama bunda juga sedih mereka tulang punggung ...,"['bunda', 'sedih', 'tulang', 'punggung', 'kelu...",bunda sedih tulang punggung keluarga gmn istri...
1,? Ada aplikasi lain kan? Lebay amat,Sarcasm,False,? Ada aplikasi lain kan? Lebay amat,Ada aplikasi lain kan Lebay amat,ada aplikasi lain kan lebay amat,"['aplikasi', 'lebay']",aplikasi lebay
2,????? Lawak UMKM ug mana? Yg harus di blokir i...,Sarcasm,False,????? Lawak UMKM ug mana? Yg harus di blokir i...,Lawak UMKM mana harus blokir itu produk impor...,lawak umkm mana harus blokir itu produk impor...,"['lawak', 'umkm', 'blokir', 'produk', 'impor',...",lawak umkm blokir produk impor cina lapang ker...
3,.. dan menghacurkan UMKM !,Sarcasm,False,.. dan menghacurkan UMKM !,dan menghacurkan UMKM,dan menghacurkan umkm,"['menghacurkan', 'umkm']",menghacurkan umkm
4,.jika tidak menggunakan teknologi manusia tida...,Sarcasm,False,.jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,"['teknologi', 'manusia', 'pergi', 'angkasa', '...",teknologi manusia pergi angkasa f x x negara k...
...,...,...,...,...,...,...,...,...
42931,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Non-Sarcasm,False,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Aaa makasii banyak kak penjelasannya penyampai...,aaa makasii banyak kak penjelasannya penyampai...,"['aaa', 'makasii', 'kak', 'penjelasannya', 'pe...",aaa makasii kak jelas sampai mudah bgt paham l...
42932,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Non-Sarcasm,False,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Aaa makaasih masi process krn pgn product stab...,aaa makaasih masi process krn pgn product stab...,"['aaa', 'makaasih', 'masi', 'process', 'pgn', ...",aaa makaasih mas process pgn product stabil ju...
42933,Aaaa luvsss🥺🥰 semoga aku terus bisa menginspir...,Non-Sarcasm,False,Aaaa luvsss (Menggambarkan keinginan akan sesu...,Aaa luvss semoga aku terus bisa menginspirasi yaa,aaa luvss semoga aku terus bisa menginspirasi yaa,"['aaa', 'luvss', 'semoga', 'menginspirasi', 'y...",aaa luvss moga inspirasi yaa
42934,"Aaaa jelas banget cara jelasinnya, makasih kak...",Non-Sarcasm,False,"Aaaa jelas banget cara jelasinnya, makasih kak...",Aaa jelas banget cara jelasinnya makasih kak a...,aaa jelas banget cara jelasinnya makasih kak a...,"['aaa', 'banget', 'jelasinnya', 'makasih', 'ka...",aaa banget jelasinnya makasih kak subscribe


In [4]:
df["text"] == df["new text"]
similar_text = df["text"] == df["new text"]
count_similar_text = similar_text.value_counts()
count_similar_text

,count
True,39725
False,3211


## Preprocess

In [5]:
df['emoji_berulang'] = df['emoji_berulang'].apply(lambda x: 0 if x == False else 1)
df_emoji_berulang = df
df_emoji_berulang

,text,anotasi_gpt,emoji_berulang,new text,cleaning,case_folding,stopword_removal,stemming
0,",Sama bunda juga sedih ,mereka tulang punggung...",Non-Sarcasm,0,",Sama bunda juga sedih ,mereka tulang punggung...",Sama bunda juga sedih mereka tulang punggung ...,sama bunda juga sedih mereka tulang punggung ...,"['bunda', 'sedih', 'tulang', 'punggung', 'kelu...",bunda sedih tulang punggung keluarga gmn istri...
1,? Ada aplikasi lain kan? Lebay amat,Sarcasm,0,? Ada aplikasi lain kan? Lebay amat,Ada aplikasi lain kan Lebay amat,ada aplikasi lain kan lebay amat,"['aplikasi', 'lebay']",aplikasi lebay
2,????? Lawak UMKM ug mana? Yg harus di blokir i...,Sarcasm,0,????? Lawak UMKM ug mana? Yg harus di blokir i...,Lawak UMKM mana harus blokir itu produk impor...,lawak umkm mana harus blokir itu produk impor...,"['lawak', 'umkm', 'blokir', 'produk', 'impor',...",lawak umkm blokir produk impor cina lapang ker...
3,.. dan menghacurkan UMKM !,Sarcasm,0,.. dan menghacurkan UMKM !,dan menghacurkan UMKM,dan menghacurkan umkm,"['menghacurkan', 'umkm']",menghacurkan umkm
4,.jika tidak menggunakan teknologi manusia tida...,Sarcasm,0,.jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,"['teknologi', 'manusia', 'pergi', 'angkasa', '...",teknologi manusia pergi angkasa f x x negara k...
...,...,...,...,...,...,...,...,...
42931,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Non-Sarcasm,0,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Aaa makasii banyak kak penjelasannya penyampai...,aaa makasii banyak kak penjelasannya penyampai...,"['aaa', 'makasii', 'kak', 'penjelasannya', 'pe...",aaa makasii kak jelas sampai mudah bgt paham l...
42932,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Non-Sarcasm,0,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Aaa makaasih masi process krn pgn product stab...,aaa makaasih masi process krn pgn product stab...,"['aaa', 'makaasih', 'masi', 'process', 'pgn', ...",aaa makaasih mas process pgn product stabil ju...
42933,Aaaa luvsss🥺🥰 semoga aku terus bisa menginspir...,Non-Sarcasm,0,Aaaa luvsss (Menggambarkan keinginan akan sesu...,Aaa luvss semoga aku terus bisa menginspirasi yaa,aaa luvss semoga aku terus bisa menginspirasi yaa,"['aaa', 'luvss', 'semoga', 'menginspirasi', 'y...",aaa luvss moga inspirasi yaa
42934,"Aaaa jelas banget cara jelasinnya, makasih kak...",Non-Sarcasm,0,"Aaaa jelas banget cara jelasinnya, makasih kak...",Aaa jelas banget cara jelasinnya makasih kak a...,aaa jelas banget cara jelasinnya makasih kak a...,"['aaa', 'banget', 'jelasinnya', 'makasih', 'ka...",aaa banget jelasinnya makasih kak subscribe


In [6]:
def has_repeated_words(text):
    words = text.split()
    for i in range(len(words) - 1):
        if words[i] == words[i + 1]:
            return 1
    return 0
def has_repeated_chars(text):
    return 1 if re.search(r'(.)\1{2,}', text) else 0
def has_repeated_symbols(text):
    return 1 if re.search(r'([!?,.#$%&*+=\-/\\:;<>@[\]^_`{|}~])\1{1,}', text) else 0
def has_repeated_combinations(text):
    return 1 if re.search(r'([a-zA-Z]{2,})\1+', text) else 0

In [7]:
df['repeat_word'] = df['text'].apply(has_repeated_words)
df['repeat_char'] = df['text'].apply(has_repeated_chars)
df['repeat_symbol'] = df['text'].apply(has_repeated_symbols)
df['repeat_combination'] = df['text'].apply(has_repeated_combinations)
df

,text,anotasi_gpt,emoji_berulang,new text,cleaning,case_folding,stopword_removal,stemming,repeat_word,repeat_char,repeat_symbol,repeat_combination
0,",Sama bunda juga sedih ,mereka tulang punggung...",Non-Sarcasm,0,",Sama bunda juga sedih ,mereka tulang punggung...",Sama bunda juga sedih mereka tulang punggung ...,sama bunda juga sedih mereka tulang punggung ...,"['bunda', 'sedih', 'tulang', 'punggung', 'kelu...",bunda sedih tulang punggung keluarga gmn istri...,0,0,0,0
1,? Ada aplikasi lain kan? Lebay amat,Sarcasm,0,? Ada aplikasi lain kan? Lebay amat,Ada aplikasi lain kan Lebay amat,ada aplikasi lain kan lebay amat,"['aplikasi', 'lebay']",aplikasi lebay,0,0,0,0
2,????? Lawak UMKM ug mana? Yg harus di blokir i...,Sarcasm,0,????? Lawak UMKM ug mana? Yg harus di blokir i...,Lawak UMKM mana harus blokir itu produk impor...,lawak umkm mana harus blokir itu produk impor...,"['lawak', 'umkm', 'blokir', 'produk', 'impor',...",lawak umkm blokir produk impor cina lapang ker...,0,1,1,0
3,.. dan menghacurkan UMKM !,Sarcasm,0,.. dan menghacurkan UMKM !,dan menghacurkan UMKM,dan menghacurkan umkm,"['menghacurkan', 'umkm']",menghacurkan umkm,0,0,1,0
4,.jika tidak menggunakan teknologi manusia tida...,Sarcasm,0,.jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,jika tidak menggunakan teknologi manusia tida...,"['teknologi', 'manusia', 'pergi', 'angkasa', '...",teknologi manusia pergi angkasa f x x negara k...,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
42931,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Non-Sarcasm,0,Aaaa makasii banyak kak penjelasannya❤️penyamp...,Aaa makasii banyak kak penjelasannya penyampai...,aaa makasii banyak kak penjelasannya penyampai...,"['aaa', 'makasii', 'kak', 'penjelasannya', 'pe...",aaa makasii kak jelas sampai mudah bgt paham l...,0,1,0,1
42932,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Non-Sarcasm,0,Aaaa makaaasih🫶🏻🫶🏻🫶🏻 masi on process krn pgn p...,Aaa makaasih masi process krn pgn product stab...,aaa makaasih masi process krn pgn product stab...,"['aaa', 'makaasih', 'masi', 'process', 'pgn', ...",aaa makaasih mas process pgn product stabil ju...,0,1,0,1
42933,Aaaa luvsss🥺🥰 semoga aku terus bisa menginspir...,Non-Sarcasm,0,Aaaa luvsss (Menggambarkan keinginan akan sesu...,Aaa luvss semoga aku terus bisa menginspirasi yaa,aaa luvss semoga aku terus bisa menginspirasi yaa,"['aaa', 'luvss', 'semoga', 'menginspirasi', 'y...",aaa luvss moga inspirasi yaa,0,1,0,0
42934,"Aaaa jelas banget cara jelasinnya, makasih kak...",Non-Sarcasm,0,"Aaaa jelas banget cara jelasinnya, makasih kak...",Aaa jelas banget cara jelasinnya makasih kak a...,aaa jelas banget cara jelasinnya makasih kak a...,"['aaa', 'banget', 'jelasinnya', 'makasih', 'ka...",aaa banget jelasinnya makasih kak subscribe,0,1,0,0


In [9]:
df.shape

(42936, 12)